In [ ]:
# default_exp connector 
# default_cls_lvl 3

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# hide
# not a dep but we need it for show_doc

from nbdev.showdoc import show_doc

In [ ]:
# export 

import sys
import os

import time
from pprint import pformat

from io import StringIO
import configparser

import psycopg2
import psycopg2.extras as extras

import pandas as pd


In [ ]:
# export 

from analytics_toolbox.logtools import *
from analytics_toolbox.s3funcs import *

# `connector`

Connectors are pretty damn useful for those of us who love databases. 

In [ ]:
# export

class DatabaseConnection(object):
    """
    Built on psycopg2 connections, provides high level API
    for running queries and returning necessary information.
    Implemented as an object on DBConnector.
    """

    def __init__(self, credentials, log):
        self.cred = credentials
        self.log = log

    def _connect(self):
        try:
            conn = psycopg2.connect(
                host=self.cred["hostname"],
                port=self.cred["port"],
                database=self.cred["database"],
                user=self.cred["user"],
                cursor_factory=extras.RealDictCursor,
            )
            self.log.info("Connected to {}".format(self.cred["hostname"]))
            return conn
        except Exception as e:
            self.log.debug(e)

    def _execute(self, sql, commit=False):
        """
        Private function focused on handling the nitty gritty of running a psycopg2
        execute and returning the results based on "commit" boolean.
        """
        if commit == False:
            try:
                # Time the query and get results, log info
                cur = self.conn.cursor()
                t0 = time.time()
                try:
                    cur.execute(sql)
                except Exception as e:
                    self.conn.rollback()
                    cur.execute(sql)
                runtime = time.time() - t0
                resp = cur.fetchall()
                cur.close()
                detail = '{{"database" : "{0}","runtime" : {2},\n "sql" : "{1}"}}'.format(
                    self.cred["hostname"], sql, str(runtime)
                )
                self.log.info(detail)
                return resp

            except Exception as e:
                detail = '{{"database" : {0},"error" : "{2}",\n "sql" : "{1}"}}'.format(
                    self.cred["hostname"], sql, e
                )
                self.log.debug(detail)
                return e
        else:
            try:
                # Time the query and get the ROWCOUNT, log info
                cur = self.conn.cursor()
                t0 = time.time()
                try:
                    cur.execute(sql)
                except Exception as e:
                    self.conn.rollback()
                    cur.execute(sql)
                runtime = time.time() - t0
                self.conn.commit()
                resp = str(cur.rowcount)
                detail = '{{"database" : "{0}","row_count" : {2}, "runtime" : {3},\n "sql" : "{1}"}}'.format(
                    self.cred["hostname"], sql, resp, str(runtime)
                )
                self.log.info(detail)
                cur.close()
                return resp

            except Exception as e:
                detail = '{{"database" : "{0}", "error" : "{2}",\n "sql" : "{1}"}}'.format(
                    self.cred["hostname"], sql, e
                )
                self.log.debug(detail)
                return e

    def get_pg_cursor(self, cursor_type=None):
        conn = self._connect()
        return conn, conn.cursor(cursor_factory=cursor_type)

    def qry(self, sql, commit=False):
        """
        Runs a query. Creates connection if non-existent.
        """
        if hasattr(self, "conn"):
            try:
                resp = self._execute(sql, commit)
                if commit: return resp
                else: return pd.DataFrame(resp)
            except ValueError as e:
                print(sql)
                raise Exception(e)

        else:
            self.conn = self._connect()
            resp = self._execute(sql, commit)
            if commit:
                return True
            else:
                return pd.DataFrame(resp)

    def write_df_to_table(self, 
                          dataframe: pd.DataFrame,
                          create_statement: str, 
                          schema: str, 
                          tablename: str, 
                          columns: tuple,
                          db_engine: str,
                          s3_bucket: str,
                          append = True) -> int:
        """
        Writes a dataframe to the target table using the create_statement.
        Its best practice to add CREATE TABLE IF NOT EXISTS to your statement. 
        DROPS should be done seperately and explicity from this process.
        """
        # create the table if it does not exist
        if not self.qry(create_statement, commit = True):
            raise Exception(f'Error writing {schema}.{tablename} to {self.cred["hostname"]}')
        # delete table results if append does not exist
        if not append:
            self.qry(f"DELETE FROM {schema}.{tablename}", commit = True)

        if db_engine == 'postgres':
            # prepare dataframe for efficent write
            stream = StringIO()
            dataframe.to_csv(stream, sep="\t", header=False, index=False, float_format='%.0f')
            # move to beginning of stream
            stream.seek(1)
            contents = stream.getvalue()
            # create psycopg2 cursor
            cur = self.conn.cursor()
            try:
                cur.copy_from(
                    stream, 
                    f'{schema}.{tablename}', 
                    null="", 
                    columns=columns
                )
                self.conn.commit()
            except psycopg2.errors.InFailedSqlTransaction as e:
                self.conn.rollback()
                raise Exception(f'Error copying to {schema}.{tablename}. Transaction block failed.')
            # close cursor after job is complete
            cur.close()
            return True
        elif db_engine == 'redshift':
            # write dataframe to S3 bucket, also get credentials 
            bucket = S3Bucket(s3_bucket)
            bucket.write_dataframe(dataframe, f'atb-writer/{tablename}.tsv')
            access_key = bucket.aws_access_key_id
            secret_access_key = bucket.aws_secret_access_key
            # copy 
            load = f"""
            COPY {schema}.{tablename}
            FROM 's3://{s3_bucket}/atb-writer/{tablename}.tsv'
            CREDENTIALS 'aws_access_key_id={access_key};aws_secret_access_key={secret_access_key}'
            DELIMITER '\t';
            """
            self.qry(load, commit = True)
            # delete from s3 
            bucket.bucket.Object("atb-writer/{tablename}.tsv").delete()
            return True


In [ ]:
# export

class DBConnector:
    """
    Object that stores connections to many databases, which are created
    from a .ini file that is passed as an object.
    """
    def __init__(self, config_file, log=StreamLog()):
        self.config = configparser.RawConfigParser()
        self.config.read(config_file)
        self.log = log
        self.active_connections = {}
        # Run func and initialize objects on self
        self._init_database_objects()

    def _init_database_objects(self):
        criteria = ["port", "database", "user", "hostname"]
        # review sections in config for db sections
        for section in self.config.sections():
            n = self.config.options(section)
            # if the section has the criteria, store them
            if set(criteria).issubset(set(n)):
                bkt = {}
                for i in criteria:
                    p = self.config.get(section, i)
                    bkt[i] = p
                # allows db.section referencing
                setattr(
                    self, section.replace("-", ""), DatabaseConnection(bkt, self.log)
                )
                self.active_connections[section] = DatabaseConnection(bkt, self.log)
        # Check if configuration file has proper sections
        if len(self.active_connections) == 0:
            raise Exception(
                """\n
                Configuration file contained no Postgres sections. Please review
                and format like below example :
                    [dev_db]
                    hostname = dev.yourhost.com
                    port = 5432
                    database = dbname 
                    user = htpeter
                    
                For additional help, please see documentation
                    https://htpeter.github.io/analytics_toolbox/#Help-With-Postges-Connection-Setup
                """
            )
            
    def add_file_logger(log_obj: LocalLog):
        self.log = log_obj 
        return self

    def close_all_conns(self):
        counter = 0
        conns = []
        for i in self.active_connections.keys():
            adj = i.replace("-", "")
            obj = getattr(self, adj)
            try:
                obj.conn.close()
                conns.append(adj)
                counter += 1
            except Exception as e:
                pass
        print("\n\t Closed {0} connections.\n\t {1} ".format(str(counter), str(conns)))

    def __repr__(self):
        return pformat(self.active_connections, indent=4)

    def __getitem__(self, item):
        return self.active_connections[item]




In [ ]:
# export

def move_table(table, from_db, to_db):
    assert type(from_db) == DatabaseConnection
    assert type(to_db) == DatabaseConnection
    creds = {
        "from_host": from_db.cred["hostname"],
        "from_db": from_db.cred["database"],
        "from_user": from_db.cred["user"],
        "to_host": to_db.cred["hostname"],
        "to_db": to_db.cred["database"],
        "to_user": to_db.cred["user"],
        "table": table,
    }
    os.system(
        "pg_dump -t {table} -h {from_host} -U {from_user} {from_db} | psql -h {to_host} -U {to_user} {to_db}".format(
            **creds
        )
    )


class NativeTemplateEnvironment(object):
    """
    Reads files from a folder, passed during initialization, and injects values 
    from a dictionary. Usings Python native string functions.
    """

    def __init__(self, templates_folder):
        self.path = templates_folder

    def build(self, template_name, context):
        complete_path = self.path + "/" + template_name
        # In case user passes trailing / in self.path
        complete_path = complete_path.replace("//", "/")
        f = open(complete_path, "r")
        skeleton = f.read()
        f.close()
        sql = skeleton.format(**context)
        return sql